In [1]:
import pickle
import os
import sys
import pandas as pd
import numpy as np
import re

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model ,load_model


from lifelines import CoxPHFitter
from Models.RCFR_AC_NOCOMX import SetModel
from Module.DataProcessing import DataLoad
from Module.MetricsGroup import DoMetric, DoAggMetric, DoSimEval



os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"



### Data load

In [2]:
StackedData, IntToGene, TTE, EVENT, TrIndEmbeddMask, ReferencePatIDLong, ReferencePatIDShort, NormDismInd, MergedData= DataLoad()

PatIDX = StackedData[:, 0:1].astype('int')
GeneIDX = StackedData[:, 1:2].astype('int')
GeneExp = StackedData[:, 2:3]

IndN = len(np.unique(PatIDX))
FeatN = len(np.unique(GeneIDX))


## Setting for tasks
#### WeightID: W0 = for prognosis-positive gene set, W20 = for prognosis-negative gene set

In [3]:
# Parameters for post-hoc models
ModelID ='M04'
WeightID = 'W0'

EmbedSize = 50
NCL_Feat = 5
NCL_Ind = 2
AdjCosWeight_ = 1. # This parameter affects only training phases, so thus any float number can be set in the post analysis phases. 
NumGene_CL = 100

ModelList = os.listdir('./Results/')
ModelList = [i for i in ModelList if ModelID in i and WeightID  in i  and 'hdf5' in i and 'ACNOCOMX' in i]
FilePath = './Results/'

# Model structure load
RCFR_AC, LayerList = SetModel(AdjCosWeight_, NormDismInd, TrIndEmbeddMask, IndN, FeatN, ReferencePatIDLong, ReferencePatIDShort)

# Data for calculating metric
DataMetric = [MergedData, TTE, EVENT, NCL_Ind, NCL_Feat, NumGene_CL, IntToGene]

ColList = ['Model','AvgtPRate', 'AvgtAdjPRate', 'MintAdjPRate', 'AvgABSGeCohD', 'MinABSGeCohD', 'AvgABSSurvCoef', 'MinABSSurvCoef', 'AvgSurvpVal', 
           'MaxSurvpVal', 'NegExpAvgSurvpVal', 'NegExpMinSurvpVal', 'AvgNegSigRate',  'MinNegSigRate', 'AvgPosSigRate', 'MinPosSigRate','IndCentRatio']


## Procedure for Selecting best model

In [4]:
MetricTable = pd.DataFrame(columns=ColList)
InfoFeatGroupList = []

for num, model in enumerate(ModelList[:]):
    print(num)
    

    RCFR_AC.load_weights(FilePath + model)  # Model weights load
    InpInd, InpFeat, IndEmbeddWeig, IndEmbeddReferenceLong, FeatEmbeddWeig, IndCentroid, FeatCentroid, ICosCLSim, FCosCLSim = LayerList
        
    # Metric calculation: InfoFeatGroup will be used in UMAP analysis
    metrics, InfoFeatGroup = DoMetric (DataMetric, [InpInd, InpFeat, IndEmbeddWeig, FeatEmbeddWeig, IndCentroid, FeatCentroid, ICosCLSim, FCosCLSim])
    InfoFeatGroupList.append(InfoFeatGroup)
    print(InfoFeatGroup)
    MetricTable = pd.concat([MetricTable, pd.DataFrame([[model] + metrics], columns=ColList)], axis=0)


MetricTable['GroupM'] = np.array([re.findall('.\d+', i)[1][1:] for i in  MetricTable['Model']])
MetricTable['EpNum'] = np.array([ re.findall('.\d+\.', i)[0][1:-1] for i in  MetricTable['Model']]).astype('int')
MetricTable = MetricTable.sort_values(['GroupM','EpNum'])

MetricTable.to_csv('./MetricTables/RCFR_ACNOCOMXMetricTable_'+str(WeightID)+'_Filt'+str(NumGene_CL)+'.csv',index=False)

0
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0., 0., 0., 0.]), array([0., 0., 0., 0.]), [0.572, 0.548, 0.59, 0.533]]
1
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.   , 0.008, 0.   , 0.002]), array([0.   , 0.   , 0.   , 0.012]), [0.882, 0.243, 0.674, 0.95]]
2
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.376, 0.916, 0.064, 0.14 ]), array([0.   , 0.002, 0.   , 0.028]), [0.438, 0.089, 0.059, 0.122]]
3
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.186, 0.958, 0.   , 0.258]), array([0.   , 0.002, 0.   , 0.088]), [0.0, 0.0, 0.0, 0.0]]
4
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.03 , 0.776, 0.   , 0.168]), array([0.   , 0.006, 0.   , 0.204]), [0.0, 0.0, 0.0, 0.0]]
5
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.038, 0.766, 0.   , 0.138]), array([0.   , 0.006, 0.   , 0.216])

c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1666: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1720: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1722: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1727: RuntimeWarning: divide by zero encountered in log
  log_lik = log_lik + dot(x_death_sum, beta) + weighted_average * log(denom).sum()
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1714: RuntimeWarning: invalid value encountered in multiply
  denom = 1.0 / (risk_phi - increasing_proportion * tie_phi)
c:\users\user\anaconda3\envs\ai\lib\site-package

0
100
200
300
400
0
100
200
300
400
[array([0.004, 0.   , 0.   , 0.032]), array([0.384, 0.948, 0.312, 0.796]), [0.0, 0.0, 1.0, 0.0]]
82
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.002, 0.   , 0.   , 0.026]), array([0.2  , 0.928, 0.   , 0.77 ]), [0.0, 0.0, 0.0, 0.0]]
83
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.008, 0.   , 0.   , 0.026]), array([0.228, 0.93 , 0.   , 0.728]), [0.068, 0.0, 0.0, 0.0]]
84
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.002, 0.   , 0.   , 0.04 ]), array([0.224, 0.904, 0.   , 0.702]), [0.034, 0.0, 0.0, 0.0]]
85
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.002, 0.   , 0.   , 0.028]), array([0.222, 0.92 , 0.   , 0.744]), [0.024, 0.0, 0.0, 0.0]]
86
0
100
200
300
400
0
100
200
300
400


c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1666: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1718: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / np.array([risk_phi])
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1720: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1722: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1714: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / (risk_phi - increasing_proportion * tie_phi)
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.p

0
100
200
300
400
0
100
200
300
400
[array([0.00408163, 0.        , 0.        , 0.03469388]), array([0.26938776, 0.90204082, 0.        , 0.74285714]), [0.024, 0.002, 1.0, 0.0]]
87
0


...

0
100
200
300
400
0
100
200
300
400
[array([0.002, 0.   , 0.   , 0.034]), array([0.586, 0.906, 0.282, 0.762]), [0.0, 0.004, 1.0, 0.0]]
389
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.004, 0.   , 0.   , 0.034]), array([0.688, 0.924, 0.544, 0.79 ]), [0.0, 0.003, 0.0, 0.0]]
390
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.00421941, 0.        , 0.00421941, 0.03586498]), array([0.72151899, 0.85232068, 0.60337553, 0.71940928]), [0.0, 0.004, 0.0, 0.0]]
391
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.00424628, 0.        , 0.        , 0.04883227]), array([0.76008493, 0.90870488, 0.78343949, 0.72611465]), [0.0, 0.007, 0.0, 0.0]]
392
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.00229358, 0.01146789, 0.13073394, 0.05963303]), array([0.39908257, 0.13073394, 0.53669725, 0.63990826]), [0.03, 0.143, 0.0, 0.0]]
393
0
100
200
300
0
100
200
300
0
100
200
30

c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1666: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1718: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / np.array([risk_phi])
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1720: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1722: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1714: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / (risk_phi - increasing_proportion * tie_phi)
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.p

100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.002, 0.   , 0.   , 0.024]), array([0.208, 0.942, 0.   , 0.756]), [1.0, 0.0, 0.0, 0.0]]
486
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.002, 0.   , 0.   , 0.036]), array([0.24 , 0.918, 0.   , 0.75 ]), [0.017, 0.015, 0.0, 0.0]]
487
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.002, 0.   , 0.   , 0.03 ]), array([0.44 , 0.942, 0.084, 0.776]), [0.004, 0.008, 0.0, 0.0]]
488
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.008, 0.   , 0.   , 0.04 ]), array([0.652, 0.914, 0.412, 0.802]), [0.0, 0.011, 0.0, 0.0]]
489
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.01 , 0.   , 0.   , 0.042]), array([0.712, 0.892, 0.63 , 0.806]), [0.0, 0.01, 0.0, 0.0]]
490
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
0
100
200
300
400
[array([0.00843882, 0.0021097 , 0.01687764, 0.03797468

c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1666: RuntimeWarning: overflow encountered in exp
  scores = weights * exp(dot(X, beta))
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1718: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / np.array([risk_phi])
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1720: RuntimeWarning: invalid value encountered in multiply
  a1 = risk_phi_x_x * denom
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1722: RuntimeWarning: invalid value encountered in multiply
  summand = numer * denom[:, None]
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.py:1714: RuntimeWarning: divide by zero encountered in true_divide
  denom = 1.0 / (risk_phi - increasing_proportion * tie_phi)
c:\users\user\anaconda3\envs\ai\lib\site-packages\lifelines\fitters\coxph_fitter.p

100
200
300
[array([0.08253968, 0.2031746 , 0.        , 0.14285714]), array([0.0031746 , 0.        , 0.        , 0.08888889]), [0.285, 0.0, 0.0, 1.0]]
499
0
100
200
300
0
100
200
300
0
100
200
300
0
100
200
300
[array([0.08917197, 0.69426752, 0.        , 0.14968153]), array([0.        , 0.        , 0.        , 0.21974522]), [0.003, 0.0, 0.0, 0.0]]


## Priority-based filter out by metrics

In [4]:
def Aggregation(MetricTable,AggMetricList):
    AggMetricTable = DoSimEval(MetricTable, 'MaxSurvpVal',pCutoff, AggMetricList, ExcRate, NmodEahG)
    AggMetricRank = DoAggMetric(AggMetricList, AggMetricTable[['Model']+AggMetricList]).sort_values('Metrics')
    AggMetricRank = pd.merge(AggMetricRank, AggMetricTable[['Model','MaxSurvpVal']], on='Model', how='left')
    BestModel = AggMetricRank.sort_values('Metrics').iloc[-1]
    
    return AggMetricRank, BestModel

In [5]:
pCutoff = 0.005
ExcRate = 0.2
NmodEahG = 1

NegMetricList = ['IndCentRatio', 'MinABSSurvCoef', 'AvgABSSurvCoef',  'MinNegSigRate', 'AvgNegSigRate', 'MinABSGeCohD', 'AvgABSGeCohD']
PosMetricList = ['IndCentRatio', 'MinABSSurvCoef', 'AvgABSSurvCoef', 'MinPosSigRate', 'AvgPosSigRate', 'MinABSGeCohD', 'AvgABSGeCohD']

MetricTable = pd.read_csv('./MetricTables/RCFR_ACNOCOMXMetricTable_'+str(WeightID)+'_Filt'+str(NumGene_CL)+'.csv')
MetricTable['GroupM'] = np.array([re.findall('.\d+', i)[2][1:] for i in  MetricTable['Model']])
MetricTable['EpNum'] = np.array([ re.findall('.\d+\.', i)[0][1:-1] for i in  MetricTable['Model']]).astype('int')




In [6]:
NegAggMetricRank, NegBestModel =  Aggregation(MetricTable, NegMetricList)
PosAggMetricRank, PosBestModel =  Aggregation(MetricTable, PosMetricList)

NegAggMetricRank.to_csv('./MetricTables/RCFR_ACNOCOMX_Neg_AggMetricRank_'+str(WeightID)+'_Filt'+str(NumGene_CL)+'.csv',index=False)
PosAggMetricRank.to_csv('./MetricTables/RCFR_ACNOCOMX_Pos_AggMetricRank_'+str(WeightID)+'_Filt'+str(NumGene_CL)+'.csv',index=False)

1
N obs with filter of IndCentRatio : 46
N obs with filter of MinABSSurvCoef : 37
N obs with filter of AvgABSSurvCoef : 30
N obs with filter of MinNegSigRate : 24
N obs with filter of AvgNegSigRate : 20
N obs with filter of MinABSGeCohD : 16
N obs with filter of AvgABSGeCohD : 13

2
N obs with filter of IndCentRatio : 46
N obs with filter of MinABSSurvCoef : 37
N obs with filter of AvgABSSurvCoef : 30
N obs with filter of MinNegSigRate : 24
N obs with filter of AvgNegSigRate : 20
N obs with filter of MinABSGeCohD : 16
N obs with filter of AvgABSGeCohD : 13

3
N obs with filter of IndCentRatio : 48
N obs with filter of MinABSSurvCoef : 39
N obs with filter of AvgABSSurvCoef : 32
N obs with filter of MinNegSigRate : 26
N obs with filter of AvgNegSigRate : 21
N obs with filter of MinABSGeCohD : 17
N obs with filter of AvgABSGeCohD : 14

4
N obs with filter of IndCentRatio : 50
N obs with filter of MinABSSurvCoef : 40
N obs with filter of AvgABSSurvCoef : 32
N obs with filter of MinNegSigR

In [13]:
NegAggMetricRank[NegMetricList].describe()

,IndCentRatio,MinABSSurvCoef,AvgABSSurvCoef,MinNegSigRate,AvgNegSigRate,MinABSGeCohD,AvgABSGeCohD
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.477346,2.618081,3.850356,0.080000,0.313100,0.094230,0.190996
std,0.040943,0.783046,0.587202,0.008246,0.065875,0.013861,0.036210
min,0.432494,1.761047,3.207171,0.074000,0.265000,0.083979,0.164102
25%,0.432494,1.761047,3.207171,0.074000,0.265000,0.084176,0.164749
50%,0.507246,3.135969,4.265779,0.074000,0.265000,0.084176,0.164815
75%,0.507246,3.216172,4.285829,0.088000,0.383500,0.108969,0.229922
max,0.507246,3.216172,4.285829,0.090000,0.387000,0.109851,0.231391


In [14]:
PosAggMetricRank[PosMetricList].describe()

,IndCentRatio,MinABSSurvCoef,AvgABSSurvCoef,MinPosSigRate,AvgPosSigRate,MinABSGeCohD,AvgABSGeCohD
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.443947,2.395261,3.269293,0.223151,0.482725,0.114261,0.209207
std,0.002741,0.069545,0.049308,0.009574,0.001927,0.000979,0.001893
min,0.439446,2.301132,3.228068,0.213115,0.480186,0.113286,0.205921
25%,0.444444,2.375336,3.244146,0.216783,0.481265,0.113620,0.209328
50%,0.444444,2.397926,3.247830,0.219114,0.483645,0.113806,0.209914
75%,0.444444,2.406925,3.274002,0.233100,0.483683,0.115050,0.210379
max,0.446957,2.494988,3.352421,0.233645,0.484848,0.115542,0.210495
